In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizerFast

In [ ]:
class NERDataset(Dataset):
    def __init__(self, texts, tags, tokenizer, tag2idx, max_len=512):
        """
        Инициализация датасета для задачи NER.

        :param texts: Список текстов (предложений).
        :param tags: Список списков меток для каждого текста.
        :param tokenizer: Быстрый токенизатор для преобразования текста в токены.
        :param tag2idx: Словарь сопоставления меток с индексами.
        :param max_len: Максимальная длина последовательности.
        """
        self.texts = texts
        self.tags = tags
        self.tokenizer = tokenizer
        self.tag2idx = tag2idx
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        tags = self.tags[idx]

        # Токенизация текста с возвратом информации о разбиении слов на под-токены
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_offsets_mapping=True,  # Для выравнивания меток
            truncation=True
        )

        # Инициализация меток
        labels = []
        word_ids = encoding.word_ids()  # Требуется быстрый токенизатор

        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                labels.append(-100)  # Специальный токен, который будет проигнорирован в потере
            elif word_idx != previous_word_idx:
                # Начало нового слова
                labels.append(self.tag2idx.get(tags[word_idx], self.tag2idx["O"]))
            else:
                # Под-токен слова
                labels.append(-100)  # Игнорируем под-токены

            previous_word_idx = word_idx

        # Преобразование меток в тензор и обрезка до max_len
        labels = labels[:self.max_len]
        labels += [-100] * (self.max_len - len(labels))

        return {
            'text': text,
            'input_ids': torch.tensor(encoding['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(encoding['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

In [2]:
! git clone https://github.com/DemirShah/NER-Extension-

Cloning into 'NER-Extension-'...


In [3]:
cd REPOSITORY

[Errno 2] No such file or directory: 'REPOSITORY'
/Users/demir/NER Email


fatal: not a git repository (or any of the parent directories): .git


In [5]:
! git init

Initialized empty Git repository in /Users/demir/NER Email/.git/


In [7]:
! git add "Custom Dataset.ipynb"

In [8]:
! git commit -m "Добавление Custom Dataset.ipynb"
! git push origin main  # или название вашей ветки
   

[main (root-commit) 6a337d6] Добавление Custom Dataset.ipynb
 Committer: Demir <demir@MacBook-Pro-Demir-2.local>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly. Run the
following command and follow the instructions in your editor to edit
your configuration file:

    git config --global --edit

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 1 file changed, 207 insertions(+)
 create mode 100644 Custom Dataset.ipynb
fatal: 'origin' does not appear to be a git repository
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [9]:
! git remote add origin https://github.com/DemirShah/NER-Extension-
   

In [10]:
! git remote -v
   

origin	https://github.com/DemirShah/NER-Extension- (fetch)
origin	https://github.com/DemirShah/NER-Extension- (push)


In [ ]:
! git push -u origin main
   

Username for 'https://github.com': 

In [12]:
! git config --global user.name "DemirShah"
! git config --global user.email "d.goog@yandex.ru"